<a href="https://colab.research.google.com/github/ProjektVek/Prova_CNC/blob/main/prova_cnc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importando or-tools para resolução de equação lineares:

In [ ]:
!pip install ortools
from ortools.linear_solver import pywraplp

Criação do Resolvedor:

In [2]:
solver = pywraplp.Solver.CreateSolver("GLOP")

Importando e lendo CSV:

In [3]:
import csv

folder = "drive/MyDrive/UNIP/prova-cnc/"
data_file = folder + "data.csv"
nutrientes_file = folder + "nutrientes.csv"

print("\nNutrientes:")
with open(nutrientes_file,'r') as data:
   for line in csv.reader(data):
            print(line)

print("\nDados:")
with open(data_file,'r') as data:
   for line in csv.reader(data):
            print(line)            


Nutrientes:
['nome', 'minimo']
['Calories (kcal)', '3.0']
['Protein (g)', '70.0']
['Calcium (g)', '0.8']
['Iron (mg)', '12.0']
['Vitamin A (KIU)', '5.0']
['Vitamin B1 (mg)', '1.8']
['Vitamin B2 (mg)', '2.7']
['Niacin (mg)', '18.0']
['Vitamin C (mg)', '75.0']

Dados:
['ingrediente', 'quantidade', 'preco', 'Calories (kcal)', 'Protein (g)', 'Calcium (g)', 'Iron (mg)', 'Vitamin A (KIU)', 'Vitamin B1 (mg)', 'Vitamin B2 (mg)', 'Niacin (mg)', 'Vitamin C (mg)']
['Wheat Flour (Enriched)', '10 lb.', '36.0', '44.7', '1411', '2.0', '365', '0.0', '55.4', '33.3', '441', '0']
['Macaroni', '1 lb.', '14.1', '11.6', '418', '0.7', '54', '0.0', '3.2', '1.9', '68', '0']
['Wheat Cereal (Enriched)', '28 oz.', '24.2', '11.8', '377', '14.4', '175', '0.0', '14.4', '8.8', '114', '0']
['Corn Flakes', '8 oz.', '7.1', '11.4', '252', '0.1', '56', '0.0', '13.5', '2.3', '68', '0']
['Corn Meal', '1 lb.', '4.6', '36.0', '897', '1.7', '99', '30.9', '17.4', '7.9', '106', '0']
['Hominy Grits', '24 oz.', '8.5', '28.6', '68

Transferindo conteúdo dos arquivos para listas:

In [4]:
nutrientes = []
with open(nutrientes_file,'r') as data:
   for line in csv.reader(data):
            nutrientes.append(line)

dados = []
with open(data_file,'r') as data:
   for line in csv.reader(data):
            dados.append(line)

Separando os dados por colunas:



In [5]:
columns = [[] for item in dados[0]]

for i, list in enumerate(columns):
    for line in dados:
      list.append(line[i])
    columns[i] = list

Adicionando Variáveis no resolvedor:


In [6]:
print('Número de Restrições =', solver.NumConstraints())
variables = []
for name in columns[0]:
  if name != 'ingrediente':
    variables.append(solver.NumVar(0,solver.infinity(), name))
print('Número de Restrições =', solver.NumConstraints())

Número de Restrições = 0
Número de Restrições = 0


Adicionando Restrições:

In [7]:
for e, line in enumerate(nutrientes):
  if e > 0:
    c_name = f'restricao_({line[0]})'
    minimum = float(line[1])
    c = solver.Constraint(minimum,solver.infinity(),c_name)
    for coluna in columns:
      if coluna[0] == line[0]:
        for var, coef in zip(variables, coluna[1:]):
          c.SetCoefficient(var, float(coef))
print('Número de Restrições =', solver.NumConstraints())

Número de Restrições = 9


Criando o objetivo:

In [8]:
objective = solver.Objective()
obj_price = columns[2][1:]

for var, obj in zip(variables, obj_price):
  objective.SetCoefficient(var, float(obj))
objective.SetMinimization()

Exportando como modelo:

In [12]:
export = solver.ExportModelAsLpFormat(False)
print(export)

\ Generated by MPModelProtoExporter
\   Name             : 
\   Format           : Free
\   Constraints      : 9
\   Variables        : 77
\     Binary         : 0
\     Integer        : 0
\     Continuous     : 77
Minimize
 Obj: +36 Wheat_Flour_(Enriched) +14.1 Macaroni +24.2 Wheat_Cereal_(Enriched) +7.1 Corn_Flakes +4.6 Corn_Meal +8.5 Hominy_Grits +7.5 Rice +7.1 Rolled_Oats +7.9 White_Bread_(Enriched) +9.1 Whole_Wheat_Bread +9.1 Rye_Bread +24.8 Pound_Cake +15.1 Soda_Crackers +11 Milk +6.7 Evaporated_Milk_(can) +30.8 Butter +16.1 Oleomargarine +32.6 Eggs +24.2 Cheese_(Cheddar) +14.1 Cream +17.9 Peanut_Butter +16.7 Mayonnaise +20.3 Crisco +9.8 Lard +39.6 Sirloin_Steak +36.4 Round_Steak +29.2 Rib_Roast +22.6 Chuck_Roast +14.6 Plate +26.8 Liver_(Beef) +27.6 Leg_of_Lamb +36.6 Lamb_Chops_(Rib) +30.7 Pork_Chops +24.2 Pork_Loin_Roast +25.6 Bacon +27.4 Ham,_smoked +16 Salt_Pork +30.3 Roasting_Chicken +42.3 Veal_Cutlets +13 Salmon,_Pink_(can) +4.4 Apples +6.1 Bananas +26 Lemons +30.9 Oranges +

Resolvendo o Sistema:

In [14]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
  print('solução:')
  print('Valor do Objetivo: ', solver.Objective().Value(),'\n')
  for variavel in variables:
    print(f'{variavel} = ', variavel.solution_value())
else:
  print('o problema não possui solução ótima')

solução:
Valor do Objetivo:  0.7164607604834595 

Wheat Flour (Enriched) =  0.0
Macaroni =  0.0
Wheat Cereal (Enriched) =  0.0
Corn Flakes =  0.0
Corn Meal =  0.005344246335991804
Hominy Grits =  0.0
Rice =  0.0
Rolled Oats =  0.0
White Bread (Enriched) =  0.0
Whole Wheat Bread =  0.0
Rye Bread =  0.0
Pound Cake =  0.0
Soda Crackers =  0.0
Milk =  0.0
Evaporated Milk (can) =  0.0
Butter =  0.0
Oleomargarine =  0.0
Eggs =  0.0
Cheese (Cheddar) =  0.0
Cream =  0.0
Peanut Butter =  0.0
Mayonnaise =  0.0
Crisco =  0.0
Lard =  0.0
Sirloin Steak =  0.0
Round Steak =  0.0
Rib Roast =  0.0
Chuck Roast =  0.0
Plate =  0.0
Liver (Beef) =  0.0
Leg of Lamb =  0.0
Lamb Chops (Rib) =  0.0
Pork Chops =  0.0
Pork Loin Roast =  0.0
Bacon =  0.0
Ham, smoked =  0.0
Salt Pork =  0.0
Roasting Chicken =  0.0
Veal Cutlets =  0.0
Salmon, Pink (can) =  0.0
Apples =  0.0
Bananas =  0.0
Lemons =  0.0
Oranges =  0.0
Green Beans =  0.0
Cabbage =  0.01131324508827593
Carrots =  0.0
Celery =  0.0
Lettuce =  0.0
Onio